# AI cập nhật tin tức mới nhất cho báo cáo tuần

In [1]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import gemini_analyze_fuction
import gemini_comment_fuction
import gemini_setup_function
import gemini_summary_function
import get_and_crawl_data
import plotly_and_upload

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(gemini_analyze_fuction)
importlib.reload(gemini_comment_fuction)
importlib.reload(gemini_setup_function)
importlib.reload(gemini_summary_function)
importlib.reload(get_and_crawl_data)
importlib.reload(plotly_and_upload)

from import_default import *
from import_database import *
from import_other import *
from gemini_analyze_fuction import *
from gemini_comment_fuction import *
from gemini_setup_function import *
from gemini_summary_function import *
from get_and_crawl_data import *
from plotly_and_upload import *

In [2]:
# Thiết lập kết nối với cơ sở dữ liệu
genai.configure(api_key=load_env("GEMINI_API"))

# Sắp xếp danh sách model theo thứ tự ổn định
# fast_model_list = select_fast_models(get_gemini_models())
# standard_model_list = select_standard_models(get_gemini_models())
fast_model_list = ['gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']
standard_model_list = ['gemini-2.5-flash', 'gemini-2.0-flash', 'gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']

# Tạo dictionary cho các model
fast_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in fast_model_list}
standard_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in standard_model_list}

In [3]:
article_url_dict = {
    'VietStock': {
        'https://vietstock.vn/4222/bat-dong-san/du-an.rss': 10,
        'https://vietstock.vn/757/tai-chinh/ngan-hang.rss': 10,
        'https://vietstock.vn/775/the-gioi/kinh-te-nganh.rss': 10,
        'https://vietstock.vn/768/kinh-te/kinh-te-dau-tu.rss': 10,
        'https://vietstock.vn/830/chung-khoan/co-phieu.rss': 10,
        'https://vietstock.vn/761/kinh-te/vi-mo.rss': 10,
        'https://vietstock.vn/772/the-gioi/tai-chinh-quoc-te.rss': 10,
        'https://vietstock.vn/773/the-gioi/chung-khoan-the-gioi.rss': 10,
        'https://vietstock.vn/737/doanh-nghiep/hoat-dong-kinh-doanh.rss': 10,
    },
    'CafeF': {
        'https://cafef.vn/vi-mo-dau-tu.chn': 10,
        'https://cafef.vn/thi-truong-chung-khoan.chn': 20,
        'https://cafef.vn/tai-chinh-quoc-te.chn': 20,
        'https://cafef.vn/doanh-nghiep.chn': 20,
        'https://cafef.vn/bat-dong-san.chn': 10,
        'https://cafef.vn/tai-chinh-ngan-hang.chn': 10,
    },
    'Vietnambiz': {
        'https://vietnambiz.vn/tai-chinh.htm': 20,
        'https://vietnambiz.vn/quoc-te.htm': 10,
        'https://vietnambiz.vn/chung-khoan.htm': 20,
        'https://vietnambiz.vn/doanh-nghiep.htm': 20,
    },
    'VnEconomy': {
        'https://vneconomy.vn/kinh-te-the-gioi.rss': 20,
        'https://vneconomy.vn/tai-chinh.rss': 20,
        'https://vneconomy.vn/dia-oc.rss': 20,
        'https://vneconomy.vn/chung-khoan.rss': 10,
    }
}

In [4]:
# 1. Tạo dictionary cấu hình để quản lý cả 2 pattern xử lý
source_handlers = {
    'VnEconomy': {
        'type': 'batch',
        'process_batch': get_article_vneconomy 
    },
    'VietStock': {
        'type': 'item_by_item',
        'get_articles': lambda url, num: feedparser.parse(url).entries[:num],
        'get_details': lambda entry: get_article_vietstock(entry['id']),
        'get_published_time': lambda entry: getattr(entry, 'published', '') or ''
    },
    'CafeF': {
        'type': 'item_by_item',
        'get_articles': get_cafef_articles_list,
        'get_details': lambda entry: get_article_cafef(entry['id']),
        'get_published_time': lambda entry: get_cafef_published_time(entry['id'])
    },
    'Vietnambiz': {
        'type': 'item_by_item',
        'get_articles': get_vietnambiz_articles_list,
        'get_details': lambda entry: get_article_vietnambiz(entry['id']),
        'get_published_time': lambda entry: get_vietnambiz_published_time(entry['id'])
    }
}

# 2. Vòng lặp xử lý chính, giờ đây đã trở nên gọn gàng
raw_news_list = []
for source, rss_list in article_url_dict.items():
    for rss_url, num_articles in rss_list.items():
        
        handler = source_handlers.get(source)
        if not handler:
            print(f"Warning: No handler found for source '{source}'. Skipping.")
            continue

        # Phân luồng xử lý dựa trên 'type' đã định nghĩa trong handler
        if handler['type'] == 'batch':
            # Xử lý các nguồn trả về một danh sách hoàn chỉnh
            temp_news_list = handler['process_batch'](rss_url, num_articles)
            raw_news_list.extend(temp_news_list) # Dùng extend hiệu quả hơn là +

        elif handler['type'] == 'item_by_item':
            # Xử lý các nguồn cần lấy chi tiết từng tin
            feed_entries = handler['get_articles'](rss_url, num_articles)
            for entry in feed_entries:
                content, image_url = handler['get_details'](entry)
                published_time = handler['get_published_time'](entry)
                
                raw_news_list.append({
                    'source': source,
                    'title': entry['title'], 
                    'content': content,
                    'image_url': image_url,
                    'article_url': entry['id'],
                    'published_time': published_time,
                })

# 3. Chuyển đổi danh sách tin thành DataFrame
raw_news_df = pd.DataFrame(raw_news_list)
raw_news_df['published_time'] = raw_news_df['published_time'].apply(convert_published_time)

In [5]:
# Lọc và phân loại các tin nổi bật
fithered_news_df = raw_news_df.copy()
filtered_news_index_dict = get_filtered_news_index(standard_model_dict, fithered_news_df, num_articles=20)

# Tạo dictionary ánh xạ từ tên nhóm đến index của các tin nổi bật
news_type_map = {}
for k, idx_list in filtered_news_index_dict.items():
    for idx in idx_list:
        news_type_map[idx] = k

# Thêm cột news_type vào fithered_news_df và lọc ra các dòng có news_type
fithered_news_df['news_type'] = fithered_news_df.index.map(news_type_map)
fithered_news_df = fithered_news_df[fithered_news_df['news_type'].notnull()]
fithered_news_df = fithered_news_df.sort_values(by=['news_type', 'published_time'], ascending=[False, False]).reset_index(drop=True)

# Loại tin doanh nghiệp vì báo cáo tuần ko cần
fithered_news_df = fithered_news_df[fithered_news_df['news_type'] != 'doanh_nghiep']

✅ Model 'gemini-2.0-flash' thành công lần 1/2 tại hàm get_filtered_news_index.


In [6]:
# Tạo danh sách tin hàng tuần với nội dung tóm tắt
weekly_news_list_df = fithered_news_df.copy()
weekly_news_list_df['content'] = weekly_news_list_df['content'].apply(lambda x: summary_weekly_article(fast_model_dict, x))

# Thêm các cột cần thiết
weekly_news_list_df['word_count'] = weekly_news_list_df['content'].str.split().str.len()
weekly_news_list_df['impact'] = analyze_news_impact(standard_model_dict, weekly_news_list_df)
weekly_news_list_df['sectors'] = analyze_news_sectors(standard_model_dict, weekly_news_list_df)

✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_weekly_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_weekly_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_weekly_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_weekly_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_weekly_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_weekly_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_weekly_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_weekly_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_weekly_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_weekly_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_weekly_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_weekly_article.
✅ Mo

In [7]:
# Lựa chọn tin tức nổi bật nhất cho từng nhóm
top_news_index_list = []
for news_type in weekly_news_list_df['news_type'].unique():
    temp_news_index_list = get_weekly_top_news(standard_model_dict, weekly_news_list_df, news_type, num_articles = 5)
    top_news_index_list = top_news_index_list + temp_news_index_list
weekly_news_list_df['ai_selected'] = weekly_news_list_df.index.isin(top_news_index_list)
weekly_news_list_df['ai_selected'] = weekly_news_list_df['ai_selected'].apply(lambda x: 'x' if x else '')

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm get_weekly_top_news.
✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm get_weekly_top_news.


In [8]:
# Thêm các nhận xét cho các nhóm tin tức
trong_nuoc_comment = generate_content_with_model_dict(standard_model_dict, weekly_news_comment_prompt(weekly_news_list_df, 'trong_nuoc'), 'weekly_trong_nuoc_news_comment')
quoc_te_comment = generate_content_with_model_dict(standard_model_dict, weekly_news_comment_prompt(weekly_news_list_df, 'quoc_te'), 'weekly_quoc_te_news_comment')
weekly_news_comments_df = pd.DataFrame({
    'news_type': ['trong_nuoc', 'quoc_te'],
    'comment': [trong_nuoc_comment, quoc_te_comment],
})

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm weekly_trong_nuoc_news_comment.
✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm weekly_quoc_te_news_comment.


In [9]:
%%capture
save_to_mssql(cts_engine, weekly_news_list_df, 'weekly_news_list')
save_to_mssql(cts_engine, weekly_news_comments_df, 'weekly_news_comments')